<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os, csv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
MODEL_REFRESH_STRATEGY = "RETRAIN" # Retrain model after every loop: RETRAIN, PRETRAIN
OPTIMIZE_STRATEGY = "NEWLY_ADDED" # Only optimize the newly added catalyst: NEWLY_ADDED, ALL
MODEL_FIT_USE_COLS = "models/high_corr_cols.txt" # Use all the columns: "ALL", "models/high_corr_cols.txt"
DEFAULT_PCA_DIMENSION = 4
START_SAMPLE_SIZE = 3
DATAFILE_NAME = "merged_large_catalyst.csv" # "reduced_dim_space_ddG.csv", "merged_large_catalyst.csv"
CD_STEP_SIZE = 0.001
CD_ITERATIONS_PER_STEP = 2
CD_ITERATIONS = 10

df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))
## removing ddG = 0 and indexing catalysts
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)

if DATAFILE_NAME == "reduced_dim_space_ddG.csv":
  X_COLS = ['x', 'y', 'z']
else:
  if MODEL_FIT_USE_COLS == "ALL":
    X_COLS = df.columns[~df.columns.isin(['Catalyst', 'ddG'])]
  else:
    with open(resolve_path_gdrive(MODEL_FIT_USE_COLS)) as f:
      reader = csv.reader(f)
      X_COLS = list(reader)[0][:-1]

In [ ]:
df.head()

In [ ]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts, PCA_DIMENSION=DEFAULT_PCA_DIMENSION):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=PCA_DIMENSION)
    print("PLS: Fitting model with {} rows and {} columns. PCA dimensions are {}"
        .format(len(X), len(X.columns), PCA_DIMENSION))
    # fitting the model
    return model.fit(X, y)

## Importing model created in "eda.ipynb"
from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
print(pls_large_saved_model)
## Read the dimension directly from the model so that if the model changes
## the dimension will be reflected automatically
PCA_DIMENSION = pls_large_saved_model.n_components

def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

## Add a predicted ddG column to the df
#df['pred_ddG'] = df.apply(lambda x: pls_predict_ee(pls_large_saved_model, np.array(x[X_COLS]))[0][0], axis=1)

In [ ]:
# start with a few randomly selected catalyst
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
## This will be used to track progress of coordinate descent
print("The max value (target) of the ddG is at {}. Currently we have reached {}.".format(df['ddG'].max(), random_catalyst_df['ddG'].max()))
random_catalyst_df

In [ ]:
original_ee = df['ddG']
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=CD_ITERATIONS,
                       cd_iterations=CD_ITERATIONS_PER_STEP, step_size=CD_STEP_SIZE):
    if pls_model == None:
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts


In [ ]:
if MODEL_REFRESH_STRATEGY == "RETRAIN":
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS])
else:
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

In [ ]:
# Perform k-nearest neighbors analysis
def recomputeKNN(df, removePointsDF = None):
  if len(removePointsDF) > 0:
    df.drop(removePointsDF.index, axis=0, inplace=True)
  num_neighbors = 1  # Number of neighbors to consider
  knn = NearestNeighbors(n_neighbors=num_neighbors)
  print('KNN: Fitting model with {} points.'.format(len(df)))
  knn.fit(df)  # Using all the catalysts
  return knn

knn = recomputeKNN(df[X_COLS], random_catalyst_df)

In [ ]:
## Recursive Loop
## The method will get unique neighbors that are not already in the list.
## If the 1st neighbor is already in the list then we try the 2nd, 3rd and so on
## till a unique neighbor is found
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        ## Add found catalysts into our batch (random_catalyst_df)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

In [ ]:
# We will now do all this in a single loop

## Maximum of entire dataset
max_ddG = df['ddG'].max()
print("The max value (target) of the ddG is {}. Model training strategy is {}".format(max_ddG, MODEL_REFRESH_STRATEGY))

## Get random rows
#random_catalyst_df = df.loc[not_highest_index].sample(START_SAMPLE_SIZE)
## Get lowest ddG to start
#random_catalyst_df = df.sort_values('ddG').head(START_SAMPLE_SIZE)
## Static list to start
random_catalyst_df = df[df['ddG'] > .5].sort_values('ddG').head(START_SAMPLE_SIZE)

while(len(random_catalyst_df) < len(df)):
    print("The current maximum ddg is at: " + str(random_catalyst_df['ddG'].max()))
    what_to_optimize = random_catalyst_df[X_COLS]
    if OPTIMIZE_STRATEGY == "NEWLY_ADDED":
        what_to_optimize = what_to_optimize.tail(START_SAMPLE_SIZE)
    new_optimized_catalysts = optimize_catalysts(what_to_optimize,
        pls_large_saved_model if MODEL_REFRESH_STRATEGY == "PRETRAIN" else None)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    # recompute knn so that we dont find the same neighbors in the next loop around
    knn = recomputeKNN(df[X_COLS], random_catalyst_df)
    if random_catalyst_df['ddG'].max() >= max_ddG:
        print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
        break

In [ ]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
#pls_predict_ee(pls_large_saved_model, properties1)
# random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [ ]:
# prompt: get last 3 rows of random_catalyst_df[X_COLS]
random_catalyst_df[X_COLS].tail(3)
